# Open AI API Chat
The aim of this notebook is to explore the creation of memories using open ai models via their api.

This code should be run in the conda environment: reflect

In [1]:
import openai

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as infile:
        return infile.read()


openai.api_key = open_file('key_openai.txt').strip()


In [6]:
m = openai.Model.list()['data']

In [7]:
system_message = """MAIN PURPOSE
You are a Reflective Journaling chatbot, designed with the primary objective of facilitating users in their exploration of thoughts and feelings. Your main task is to act as a catalyst in their journey of self-discovery and personal growth. Remember, the overall purpose is not just to document the user's thoughts and feelings, but to support their journey towards deeper self-understanding and growth in a natural, human-like conversational tone.


BEHAVIORS AND METHODS
The following are guidelines for your behaviors and methodology of engagement.

Deep Dive: Encourage users to venture into the depths of their thoughts and emotions. Your dialogue should nudge them towards introspection, revealing layers of their psyche they might not be aware of. Ask pointed and exploratory questions, but do so in a smooth, conversational manner that feels less like an interrogation and more like a friendly chat.

Engage with Empathy: Provide validation when users express their feelings or ideas. This will help build trust and make them more comfortable sharing deeper aspects of themselves. Be aware, though, of avoiding undue affirmation of negative or unproductive thinking patterns.

Reframing and Reflection: When you detect unhelpful thought patterns, guide the user towards reframing their perspective. Do not impose a new frame, but gently nudge them to see the situation from different angles. Take note of recurring themes or patterns in their entries and reflect on them.

Educate and Enlighten: Where appropriate, introduce new concepts, techniques, or information that may help the user better understand their emotions and experiences. This should be done in a non-intrusive way, embedded naturally within the conversation.

The Core Issue: Your goal isn't to simply hear the user's thoughts, but to help them uncover the core issues driving their feelings and behavior. Read between the lines, use your understanding of their past entries to discern underlying themes, and gently lead them towards these revelations.

Read Between The Lines: Use your ability to infer what is going on to see the bigger picture and read between the lines. If you perceive that the user may not be focusing the most emotionally salient topic, call their attention to the broader range of emotional content. The reason for this is that not all users are fully emotionally literate, or they may be in a sub-optimal state. 

Natural Flow: The overall tone of the conversation should be easy-going, natural, and conversational. Avoid blunt, robotic responses. Do not use lists ever. Instead, aim for subtlety, nuance, and a gentle, guiding style.

Ask Questions: Whether you ask probing questions or leading questions, you should use questions as much as possible to solicit deeper reflection. Make observations and connect dots, and ask if the user noticed the patterns."""

In [8]:
# call the model with some test code
conversation = list()
conversation.append({'role': 'user', 'content': 'Explain the difference between a hybrid car and an electric car.'})
conversation.append({'role': 'system', 'content': system_message})

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=conversation, temperature=0)
text = response['choices'][0]['message']['content']


In [12]:
import textwrap
def fmt(response):
    formatted_lines = [textwrap.fill(line, width=120, initial_indent='    ', subsequent_indent='    ') for line in response.split('\n')]
    return '\n'.join(formatted_lines)


In [13]:
print(fmt(text))

    Sure! I'd be happy to explain the difference between a hybrid car and an electric car.

    A hybrid car is a vehicle that combines two different power sources to propel itself. Typically, a hybrid car has
    both an internal combustion engine (usually fueled by gasoline) and an electric motor. The electric motor is powered
    by a battery that is charged through regenerative braking or by the internal combustion engine. The main purpose of
    a hybrid car is to improve fuel efficiency and reduce emissions by using the electric motor for low-speed driving or
    when extra power is needed.

    On the other hand, an electric car, also known as an electric vehicle (EV), is powered solely by an electric motor
    and a battery pack. It does not have an internal combustion engine and does not require gasoline or any other fossil
    fuel. Electric cars are charged by plugging them into an electrical outlet or a charging station. They rely entirely
    on electricity to operate and 

In [4]:
print(response.keys())
print(response['id'])
print(response['object'])
print(response['created'])
print(response['model'])
print(len(response['choices']))
print(response['choices'][0].keys())
print(response['choices'][0]['index'])
print(response['choices'][0]['finish_reason'])
print(response['usage'].keys())
print(response['usage']['prompt_tokens'])
print(response['usage']['completion_tokens'])
print(response['usage']['total_tokens'])


dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])
chatcmpl-7knNtaLQWSkNf3Wncqbih63wzg7XP
chat.completion
1691387277
gpt-3.5-turbo-0613
1
dict_keys(['index', 'message', 'finish_reason'])
0
function_call
dict_keys(['prompt_tokens', 'completion_tokens', 'total_tokens'])
56
7
63


In [7]:
import datetime

def linux_time_to_date(linux_time):
    date_time = datetime.datetime.fromtimestamp(linux_time)
    return date_time.strftime('%d-%m-%Y %H:%M:%S')

print(linux_time_to_date(response['created']))

07-08-2023 05:47:57


There are two ways to do things. We could get the chatbot to make callbacks to invoke memories or we could design a prompt that extracts the types of memories that we want to recall.
Lets try both.

First let's get it to answer a simple question that will require a callback: What is today's date?

In [17]:
conversation = list()
conversation.append({'role': 'user', 'content': "What is today's date?"})
conversation.append({'role': 'system', 'content': "You are a helpful assistant."})

functions = list()
functions.append({'name' : 'GetCurrentDateTime', 'description': "Get's the current date and time.", 'parameters': {'type': 'object', 'properties': {}}})

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=conversation, functions=functions, temperature=0)
text = response['choices'][0]['message']['content']




In [6]:
print(response['choices'][0]['message']['function_call'])


{
  "name": "GetCurrentDateTime",
  "arguments": "{}"
}


In [18]:
def GetCurrentDateTime():
    return str(datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S'))

extend_conversation = list()
extend_conversation.extend(conversation)
extend_conversation.append(response['choices'][0]['message'])
extend_conversation.append({ 'role' : 'function', 'name' : 'GetCurrentDateTime', 'content' : GetCurrentDateTime() })

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=extend_conversation, temperature=0)
text = response['choices'][0]['message']['content']


In [11]:
datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')

'07-08-2023 06:13:40'

OK - that worked. Lets make it more specific to the problem we are addressing.

In [25]:
conversation = list()
conversation.append({'role': 'user', 'content': "What did we speak about yesterday?"})
conversation.append({'role': 'system', 'content': "You are a helpful assistant."})

functions = list()
functions.append({'name' : 'GetCurrentDateTime', 'description': "Get's the current date and time.", 'parameters': {'type': 'object', 'properties': {}}})
functions.append({'name' : 'RecallMemory', 'description': "Recalls the conversation at a given time formatted as json.", 'parameters': {
    'type': 'object', 'properties': { 
        'time' : { 
            'type': 'string',
            'description': 'The time formatted as mm:dd:yyyy hh:mm:ss.'
            }
        }
    }})

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=conversation, functions=functions, temperature=0)

In [26]:
print(response.keys())
print(response['id'])
print(response['object'])
print(response['created'])
print(response['model'])
print(len(response['choices']))
print(response['choices'][0].keys())
print(response['choices'][0]['index'])
print(response['choices'][0]['finish_reason'])
print(response['choices'][0]['message']['function_call'])
print(response['usage'].keys())
print(response['usage']['prompt_tokens'])
print(response['usage']['completion_tokens'])
print(response['usage']['total_tokens'])


dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])
chatcmpl-7koCUircdL8syt0yJNHDrv3UTG8b7
chat.completion
1691390414
gpt-3.5-turbo-0613
1
dict_keys(['index', 'message', 'finish_reason'])
0
function_call
{
  "name": "GetCurrentDateTime",
  "arguments": "{}"
}
dict_keys(['prompt_tokens', 'completion_tokens', 'total_tokens'])
96
7
103


In [29]:
c = list()
c.extend(conversation)
c.append(response['choices'][0]['message'])
c.append({ 'role' : 'function', 'name' : 'GetCurrentDateTime', 'content' : GetCurrentDateTime() })

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=c, functions=functions, temperature=0)


In [31]:
print(response.keys())
print(response['id'])
print(response['object'])
print(response['created'])
print(response['model'])
print(len(response['choices']))
print(response['choices'][0].keys())
print(response['choices'][0]['index'])
print(response['choices'][0]['finish_reason'])
print(response['choices'][0]['message']['function_call'])
print(response['usage'].keys())
print(response['usage']['prompt_tokens'])
print(response['usage']['completion_tokens'])
print(response['usage']['total_tokens'])


dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])
chatcmpl-7koDL3hoXsLdiPRlrMbI56T9VanXT
chat.completion
1691390467
gpt-3.5-turbo-0613
1
dict_keys(['index', 'message', 'finish_reason'])
0
function_call
{
  "name": "RecallMemory",
  "arguments": "{\n  \"time\": \"07-07-2023\"\n}"
}
dict_keys(['prompt_tokens', 'completion_tokens', 'total_tokens'])
129
21
150


In [32]:
d = list()
d.extend(conversation)
d.append(response['choices'][0]['message'])
d.append({ 'role' : 'function', 'name' : 'RecallMemory', 'content' : "{ 'time' : '07-07-2023 12:30:00', 'topic' : 'Electric Vehicles' }" })

response = openai.ChatCompletion.create(model='gpt-3.5-turbo-0613', messages=d, functions=functions, temperature=0)


In [34]:
print(response.keys())
print(response['id'])
print(response['object'])
print(response['created'])
print(response['model'])
print(len(response['choices']))
print(response['choices'][0].keys())
print(response['choices'][0]['index'])
print(response['choices'][0]['finish_reason'])
print(response['choices'][0]['message']['content'])
print(response['usage'].keys())
print(response['usage']['prompt_tokens'])
print(response['usage']['completion_tokens'])
print(response['usage']['total_tokens'])


dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])
chatcmpl-7koLloXmkaloo7nNAlE0Xikby1KaE
chat.completion
1691390989
gpt-3.5-turbo-0613
1
dict_keys(['index', 'message', 'finish_reason'])
0
stop
Yesterday, we spoke about electric vehicles.
dict_keys(['prompt_tokens', 'completion_tokens', 'total_tokens'])
154
9
163
